## 1. Installation & Configuration

In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from newsapi.newsapi_client import NewsApiClient
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliothèques chargées")

d:\Documents\ECE\2025\PFE\FinsightAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Bibliothèques chargées


In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path
load_dotenv(Path(".env"))
newsapi = NewsApiClient(api_key=os.getenv("NEWSAPI_API_KEY"))

# Device (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


## 2. Chargement de votre modèle FinBERT

In [5]:
# Charger votre modèle fine-tuné
MODEL_PATH = "./news_finbert_sentiment_model"

print("Chargement du modèle FinBERT...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=2,
    use_safetensors=True
)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("✓ Modèle FinBERT chargé et prêt")

Chargement du modèle FinBERT...
✓ Modèle FinBERT chargé et prêt


## 3.1. Configuration des actifs et mots-clés

### Configuration des actifs avec leurs mots-clés spécifiques

In [101]:
ASSETS_CONFIG = {
    # Indices
    'SP500': {
        'type': 'Index',
        'keywords': ['S&P 500', 'SP500', 'Wall Street', 'Dow Jones', 'Nasdaq', 'Trump', 'Fed', 'Federal Reserve', 'US economy', 'American stocks']
    },
    'CAC40': {
        'type': 'Index',
        'keywords': ['CAC 40', 'CAC40', 'Paris Bourse', 'Euronext Paris', 'Macron', 'French economy', 'France stocks']
    },
    'GER30': {
        'type': 'Index',
        'keywords': ['DAX', 'DAX 30', 'GER30', 'Frankfurt', 'German economy', 'Scholz', 'Germany stocks']
    },
    
    # Entreprises Tech
    'AAPL': {
        'type': 'Company',
        'keywords': ['Apple', 'AAPL', 'iPhone', 'iPad', 'Mac', 'Tim Cook', 'iOS', 'App Store']
    },
    'AMZN': {
        'type': 'Company',
        'keywords': ['Amazon', 'AMZN', 'AWS', 'Bezos', 'Prime', 'e-commerce', 'cloud computing']
    },
    'TSLA': {
        'type': 'Company',
        'keywords': ['Tesla', 'TSLA', 'Elon Musk', 'electric vehicle', 'EV', 'Model 3', 'Model Y', 'Cybertruck']
    },
    
    # Entreprises Françaises
    'SAN': {
        'type': 'Company',
        'keywords': ['Sanofi', 'SAN', 'pharmaceutical', 'vaccine', 'healthcare']
    },
    'CASIC': {
        'type': 'Company',
        'keywords': ['Casic', 'CASIC']
    },
    'HO': {
        'type': 'Company',
        'keywords': ['Thales', 'HO', 'defense', 'aerospace', 'cybersecurity']
    },
    'MC': {
        'type': 'Company',
        'keywords': ['LVMH', 'Louis Vuitton', 'Bernard Arnault', 'luxury', 'Dior', 'Moët Hennessy']
    },
    'ENGI': {
        'type': 'Company',
        'keywords': ['Engie', 'ENGI', 'energy', 'renewable', 'gas', 'electricity']
    },
    'TTE': {
        'type': 'Company',
        'keywords': ['TotalEnergies', 'Total', 'TTE', 'oil', 'gas', 'energy', 'petroleum']
    },
    'IHG': {
        'type': 'Company',
        'keywords': ['InterContinental Hotels', 'IHG', 'Holiday Inn', 'hotel', 'hospitality']
    },
    'AIR': {
        'type': 'Company',
        'keywords': ['Airbus', 'AIR', 'aircraft', 'A320', 'A350', 'aviation', 'aerospace']
    },
    'STLA': {
        'type': 'Company',
        'keywords': ['Stellantis', 'STLA', 'Peugeot', 'Fiat', 'Chrysler', 'Jeep', 'automotive']
    },
    
    # Commodités
    'OIL': {
        'type': 'Commodity',
        'keywords': ['crude oil', 'oil price', 'petroleum', 'Brent', 'WTI', 'OPEC', 'energy prices']
    },
    'GOLD': {
        'type': 'Commodity',
        'keywords': ['gold', 'gold price', 'precious metal', 'bullion', 'gold market']
    }
}

def build_query_from_keywords(keywords):
    """Construit une requête OR pour NewsAPI"""
    return ' OR '.join([f'"{kw}"' for kw in keywords])

print(f"✓ {len(ASSETS_CONFIG)} actifs configurés")

✓ 17 actifs configurés


## 3.2 Fonction de récupération de news

In [102]:
def fetch_financial_news(query=None, 
                         country=None, 
                         language="en",
                         page_size=100,
                         categories=None,
                         days_back=7,
                         use_asset_keywords=False):
    """
    Récupère des news financières via NewsAPI
    
    Args:
        query: Mots-clés personnalisés (ex: "Apple OR Tesla")
        country: Code pays ISO (ex: "us", "fr", "gb")
        language: Code langue ("en", "fr", etc.)
        page_size: Nombre d'articles (max 100)
        categories: Liste de catégories ['business', 'health', 'science', 'technology']
        days_back: Nombre de jours en arrière
        use_asset_keywords: Si True, utilise les mots-clés des actifs configurés
    
    Returns:
        Liste de dictionnaires avec les news + asset mapping
    """
    
    from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
    
    if categories is None:
        categories = ['business', 'health', 'science', 'technology']
    
    all_news = []
    
    # PARTIE 1: Récupération par CATÉGORIES (top-headlines)
    print("\n" + "="*80)
    print("📰 RÉCUPÉRATION PAR CATÉGORIES (Top Headlines)")
    print("="*80)
    
    for category in categories:
        try:
            print(f"\n🔍 Catégorie: {category}...")
            
            articles = newsapi.get_top_headlines(
                q=query if query and not use_asset_keywords else None,
                language=language,
                page_size=page_size,
                category=category,
                country=country
            )
            
            for article in articles.get('articles', []):
                all_news.append({
                    'title': article.get('title', ''),
                    'description': article.get('description', ''),
                    'content': article.get('content', ''),
                    'source': article.get('source', {}).get('name', 'Unknown'),
                    'url': article.get('url', ''),
                    'published_at': article.get('publishedAt', ''),
                    'author': article.get('author', 'Unknown'),
                    'news_category': category,
                    'query_keyword': 'CATEGORY_' + category.upper(),
                    'asset_ticker': None,  # Sera rempli après
                    'asset_type': None
                })
            
            print(f"   ✓ {len(articles.get('articles', []))} articles")
        
        except Exception as e:
            print(f"   ❌ Erreur: {e}")
            continue
    
    # PARTIE 2: Récupération par MOTS-CLÉS des actifs (everything)
    if use_asset_keywords:
        print("\n" + "="*80)
        print("🎯 RÉCUPÉRATION PAR ACTIFS (Mots-clés spécifiques)")
        print("="*80)
        
        for ticker, config in ASSETS_CONFIG.items():
            try:
                asset_query = build_query_from_keywords(config['keywords'])
                print(f"\n🔍 Actif: {ticker} ({config['type']})...")
                print(f"   Query: {asset_query[:100]}...")
                
                articles = newsapi.get_everything(
                    q=asset_query,
                    language=language,
                    from_param=from_date,
                    page_size=page_size,
                    sort_by='publishedAt'
                )
                
                for article in articles.get('articles', []):
                    all_news.append({
                        'title': article.get('title', ''),
                        'description': article.get('description', ''),
                        'content': article.get('content', ''),
                        'source': article.get('source', {}).get('name', 'Unknown'),
                        'url': article.get('url', ''),
                        'published_at': article.get('publishedAt', ''),
                        'author': article.get('author', 'Unknown'),
                        'news_category': 'KEYWORD_SEARCH',
                        'query_keyword': ticker,
                        'asset_ticker': ticker,
                        'asset_type': config['type']
                    })
                
                print(f"   ✓ {len(articles.get('articles', []))} articles")
            
            except Exception as e:
                print(f"   ❌ Erreur pour {ticker}: {e}")
                continue
    
    # Supprimer les doublons basés sur l'URL
    unique_news = []
    seen_urls = set()
    for news in all_news:
        if news['url'] not in seen_urls:
            seen_urls.add(news['url'])
            unique_news.append(news)
    
    print(f"\n{'='*80}")
    print(f"✅ TOTAL: {len(unique_news)} articles uniques (sur {len(all_news)} récupérés)")
    print(f"{'='*80}\n")
    
    return unique_news

In [103]:
# Ajouter après la cellule fetch_financial_news

## 3.2. Fonction de matching automatique des actifs

def match_news_to_assets(news_text):
    """
    Détecte automatiquement les actifs mentionnés dans un texte
    
    Args:
        news_text: Titre + description de la news
    
    Returns:
        Liste de tuples (ticker, type) des actifs trouvés
    """
    text_lower = news_text.lower()
    matched_assets = []
    
    for ticker, config in ASSETS_CONFIG.items():
        for keyword in config['keywords']:
            if keyword.lower() in text_lower:
                matched_assets.append((ticker, config['type']))
                break  # Un seul match par actif suffit
    
    return matched_assets

# Test
test_text = "Apple's iPhone sales surge as Tesla announces new Model Y"
matches = match_news_to_assets(test_text)
print(f"Test: '{test_text}'")
print(f"Actifs détectés: {matches}")

Test: 'Apple's iPhone sales surge as Tesla announces new Model Y'
Actifs détectés: [('AAPL', 'Company'), ('TSLA', 'Company'), ('HO', 'Company')]


## 4. Fonction d'analyse de sentiment

In [104]:
def analyze_sentiment(text, model, tokenizer, device, max_length=512):
    """
    Analyse le sentiment d'un texte avec votre modèle FinBERT
    
    Args:
        text: Texte à analyser
        model: Modèle FinBERT
        tokenizer: Tokenizer
        device: CPU ou GPU
        max_length: Longueur max (512 pour news)
    
    Returns:
        dict avec sentiment, confiance, probabilités
    """
    
    if not text or len(text.strip()) == 0:
        return {
            'sentiment': 'Unknown',
            'confidence': 0.0,
            'prob_negative': 0.5,
            'prob_positive': 0.5
        }
    
    # Tokenization
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)[0]
        
        prediction = torch.argmax(logits, dim=1).item()
        confidence = probabilities[prediction].item()
    
    sentiment_label = "Positive" if prediction == 1 else "Negative"
    
    return {
        'sentiment': sentiment_label,
        'confidence': confidence,
        'prob_negative': probabilities[0].item(),
        'prob_positive': probabilities[1].item()
    }

## 5. Pipeline complet : Fetch + Analyze

In [105]:
# Remplacer la cellule 9 (analyze_news_pipeline)

def analyze_news_pipeline(query=None, country=None, page_size=25, categories=None, use_asset_keywords=True):
    """
    Pipeline complet : récupère les news et analyse le sentiment
    
    Args:
        query: Mots-clés de recherche personnalisés
        country: Code pays ISO
        page_size: Nombre d'articles par catégorie/actif
        categories: Liste de catégories
        use_asset_keywords: Si True, boucle sur tous les actifs configurés
    
    Returns:
        DataFrame avec news + analyse + mapping actifs
    """
    
    print("="*80)
    print("🚀 PIPELINE COMPLET - NEWS + SENTIMENT + ASSET MAPPING")
    print("="*80)
    
    # 1. Récupérer les news
    news_list = fetch_financial_news(
        query=query, 
        country=country, 
        page_size=page_size,
        categories=categories,
        use_asset_keywords=use_asset_keywords
    )
    
    if not news_list:
        print("❌ Aucune news récupérée")
        return pd.DataFrame()
    
    # 2. Analyser chaque news + mapper aux actifs
    print("\n🤖 ANALYSE DE SENTIMENT + MAPPING ACTIFS")
    print("="*80)
    
    results = []
    
    for i, news in enumerate(news_list, 1):
        full_text = f"{news['title']} {news['description']}"
        
        # Analyse de sentiment
        sentiment_result = analyze_sentiment(full_text, model, tokenizer, device)
        
        # Si asset_ticker n'est pas déjà défini (cas des catégories), on le détecte
        if news['asset_ticker'] is None:
            matched_assets = match_news_to_assets(full_text)
            
            # Si aucun actif détecté, on crée une seule ligne "Unknown"
            if not matched_assets:
                matched_assets = [('UNKNOWN', 'Unknown')]
            
            # Créer une ligne pour chaque actif détecté
            for ticker, asset_type in matched_assets:
                results.append({
                    'asset_ticker': ticker,
                    'asset_type': asset_type,
                    'query_keyword': news['query_keyword'],
                    'news_category': news['news_category'],
                    'title': news['title'],
                    'description': news.get('description', ''),
                    'source': news['source'],
                    'published_at': news['published_at'],
                    'sentiment': sentiment_result['sentiment'],
                    'confidence': sentiment_result['confidence'],
                    'prob_negative': sentiment_result['prob_negative'],
                    'prob_positive': sentiment_result['prob_positive'],
                    'url': news['url']
                })
        else:
            # Asset déjà défini (cas des mots-clés)
            results.append({
                'asset_ticker': news['asset_ticker'],
                'asset_type': news['asset_type'],
                'query_keyword': news['query_keyword'],
                'news_category': news['news_category'],
                'title': news['title'],
                'description': news.get('description', ''),
                'source': news['source'],
                'published_at': news['published_at'],
                'sentiment': sentiment_result['sentiment'],
                'confidence': sentiment_result['confidence'],
                'prob_negative': sentiment_result['prob_negative'],
                'prob_positive': sentiment_result['prob_positive'],
                'url': news['url']
            })
        
        if i % 25 == 0:
            print(f"   ✓ Analysé {i}/{len(news_list)} news...")
    
    df = pd.DataFrame(results)
    
    print(f"\n{'='*80}")
    print(f"✅ {len(df)} associations news-actifs analysées")
    print(f"{'='*80}\n")
    
    return df

## 6. Exemples d'utilisation

In [ ]:
# Remplacer la cellule 11 (Exemple d'utilisation)

## 6. Exemples d'utilisation avec mapping actifs

# EXEMPLE 1: Récupération complète (catégories + actifs)
df_finance = analyze_news_pipeline(
    page_size=100,  # 100 articles par catégorie + 100 par actif
    categories=['business', 'health', 'science', 'technology'],
    use_asset_keywords=True  # Active la recherche par actifs
)

# Aperçu des résultats
print("\n📊 APERÇU DES RÉSULTATS")
print(df_finance[['asset_ticker', 'asset_type', 'query_keyword', 'title', 'sentiment', 'confidence']].head(20))

# Statistiques globales
print("\n📈 STATISTIQUES GLOBALES:")
print(f"Total news-actifs: {len(df_finance)}")
print(f"Actifs uniques: {df_finance['asset_ticker'].nunique()}")
print(f"Sources uniques: {df_finance['source'].nunique()}")

# Statistiques par actif
print("\n📊 STATISTIQUES PAR ACTIF:")
asset_stats = df_finance.groupby('asset_ticker').agg({
    'title': 'count',
    'sentiment': lambda x: (x == 'Positive').sum() / len(x) if len(x) > 0 else 0,
    'confidence': 'mean'
}).rename(columns={
    'title': 'count',
    'sentiment': 'positive_ratio',
    'confidence': 'avg_confidence'
}).sort_values('count', ascending=False)

print(asset_stats.head(15))

# Statistiques par type d'actif
print("\n📊 STATISTIQUES PAR TYPE:")
type_stats = df_finance.groupby('asset_type').agg({
    'title': 'count',
    'confidence': 'mean'
})
print(type_stats)

🚀 PIPELINE COMPLET - NEWS + SENTIMENT + ASSET MAPPING

📰 RÉCUPÉRATION PAR CATÉGORIES (Top Headlines)

🔍 Catégorie: business...
   ✓ 18 articles

🔍 Catégorie: health...
   ✓ 18 articles

🔍 Catégorie: science...
   ✓ 20 articles

🔍 Catégorie: technology...
   ✓ 20 articles

🎯 RÉCUPÉRATION PAR ACTIFS (Mots-clés spécifiques)

🔍 Actif: SP500 (Index)...
   Query: "S&P 500" OR "SP500" OR "Wall Street" OR "Dow Jones" OR "Nasdaq" OR "Trump" OR "Fed" OR "Federal Res...
   ✓ 16 articles

🔍 Actif: CAC40 (Index)...
   Query: "CAC 40" OR "CAC40" OR "Paris Bourse" OR "Euronext Paris" OR "Macron" OR "French economy" OR "France...
   ✓ 19 articles

🔍 Actif: GER30 (Index)...
   Query: "DAX" OR "DAX 30" OR "GER30" OR "Frankfurt" OR "German economy" OR "Scholz" OR "Germany stocks"...
   ✓ 18 articles

🔍 Actif: AAPL (Company)...
   Query: "Apple" OR "AAPL" OR "iPhone" OR "iPad" OR "Mac" OR "Tim Cook" OR "iOS" OR "App Store"...
   ✓ 20 articles

🔍 Actif: AMZN (Company)...
   Query: "Amazon" OR "AMZN" OR "AW

In [ ]:
# Exemple 2 : News sur des entreprises spécifiques
df_tech = analyze_news_pipeline(
    query="Apple OR Tesla OR Microsoft OR Google",
    page_size=100
)

# Filtrer par confiance élevée (> 80%)
df_high_confidence = df_tech[df_tech['confidence'] > 0.8]

print(f"\n🎯 News avec confiance > 80% : {len(df_high_confidence)} articles")
print(df_high_confidence[['title', 'sentiment', 'confidence']])

🔄 DÉBUT DU PIPELINE

📡 Récupération des news (query='Apple OR Tesla OR Microsoft OR Google')...
✓ 0 articles récupérés
❌ Aucune news récupérée


KeyError: 'confidence'

In [25]:
# Exemple 3 : Analyse par sentiment
print("\n📈 STATISTIQUES")
print(f"Sentiment positif : {(df_tech['sentiment'] == 'Positive').sum()} articles")
print(f"Sentiment négatif : {(df_tech['sentiment'] == 'Negative').sum()} articles")
print(f"\nConfiance moyenne : {df_tech['confidence'].mean():.2%}")


📈 STATISTIQUES
Sentiment positif : 52 articles
Sentiment négatif : 46 articles

Confiance moyenne : 87.96%


## 7. Export des résultats (pour intégration équipe)

In [100]:
# Sauvegarder en CSV
output_file = f"sentiment_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_finance.to_csv(output_file, index=False)
print(f"✓ Résultats sauvegardés dans : {output_file}")

# Ou en JSON pour API
output_json = output_file.replace('.csv', '.json')
df_finance.to_json(output_json, orient='records', indent=2)
print(f"✓ Résultats JSON dans : {output_json}")

✓ Résultats sauvegardés dans : sentiment_analysis_20260122_170635.csv
✓ Résultats JSON dans : sentiment_analysis_20260122_170635.json


## 8. Fonction API-ready pour votre équipe

In [33]:
def get_sentiment_analysis(query, max_results=20):
    """
    Fonction simple pour l'équipe backend/frontend
    
    Args:
        query: Requête de recherche
        max_results: Nombre max de résultats
    
    Returns:
        dict formaté pour API
    """
    df = analyze_news_pipeline(query=query, page_size=max_results)
    
    if df.empty:
        return {"status": "error", "message": "No news found", "data": []}
    
    return {
        "status": "success",
        "query": query,
        "total_articles": len(df),
        "positive_count": (df['sentiment'] == 'Positive').sum(),
        "negative_count": (df['sentiment'] == 'Negative').sum(),
        "average_confidence": float(df['confidence'].mean()),
        "articles": df.to_dict('records')
    }

# Test
result = get_sentiment_analysis("Macron", max_results=10)
print(f"\nStatus: {result['status']}")
print(f"Total: {result['total_articles']} articles")
print(f"Positif: {result['positive_count']}, Négatif: {result['negative_count']}")

🔄 DÉBUT DU PIPELINE

📡 Récupération des news (query='Macron')...
✓ 8 articles récupérés

🤖 Analyse de sentiment en cours...
   Analysé 5/8 articles...

✓ 8 news analysées avec succès

Status: success
Total: 8 articles
Positif: 4, Négatif: 4


In [34]:
for article in result['articles']:
    print(f"- {article['published_at']} {article['title']} [{article['sentiment']} - {article['confidence']:.2%}]")

- 2026-01-21T15:00:56Z Canadian PM Mark Carney Declares Flame Of American Global Era Extinguished [Positive - 81.30%]
- 2026-01-21T14:48:47Z Explainer-What is Trump’s ‘Board of Peace’ and who has joined so far? [Positive - 67.84%]
- 2026-01-21T14:48:20Z Should Scotland fans boycott the World Cup over Greenland row? [Negative - 98.06%]
- 2026-01-21T14:47:09Z Trump throws shade at French President Emmanuel Macron for his ‘beautiful’ sunglasses [Positive - 90.31%]
- 2026-01-21T14:46:46Z Trump mocks Macron’s sunglasses in Davos speech [Negative - 97.27%]
- 2026-01-21T14:43:50Z Trump on Macron’s sunglasses: ‘What the hell happened?’ [Positive - 51.05%]
- 2026-01-21T14:41:56Z Trump Mixes Up Iceland and Greenland in Incoherent Davos Speech [Negative - 97.06%]
- 2026-01-21T14:27:06Z Greenland Will Find Out [Negative - 97.24%]
